## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Étiquetage morphosyntaxique (POS Tagging) 
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Lemmatisation
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH et SNOMED présents dans les données


*Suivi des modifications apportées*  
**2022-06-17**  
- Traiter un ensemble de documents .txt plutôt qu'un seul gros corpus ✓
- Retenir seulement les termes qui ont une fréquence supérieure à x ✓
- Rammener l'étiquetage morphosyntaxique et le filtrage par patrons syntaxiques plus tôt afin de filtrer les KWIC ✓
  
**2022-06-22**  
- Ajouter le tag associé au sous-corpus dans les paths (acteur + path) ✓
- Convertir les tags de TreeTagger selon les tags utilisés par le Lefff pour les fournir au lemmatiseur (performance meilleure lorsqu'on lui indique la POS) ✓

**2022-06-23**  
- Modifier le lemmatiseur pour lui fournir les POS tags et qu'il performe mieux ✓

*Modifications suivantes*
- Étiquetage morpho-syntaxique et Lemmatisation plus tôt dans le traitement pour améliorer la performance et la rapidité des deux outils.
- En faire une fonction def nlp(corpus) pour pouvoir la relancer à l'étape de la pondération statistique ✓ (voir Notebook *3_Pondération_statistique.ipynb*)



### **Lire le corpus** 

In [74]:
import os, shutil, re
from pathlib import Path

acteur = 'inspq'
sous_corpus = False 
tag = ''

# Change the directory
if sous_corpus:
    path = '/Users/camilledemers/Documents/03-corpus/2-sous-corpus/'
    path = path + acteur + '/' + tag +'/'

else: 
    path = '/Users/camilledemers/Documents/03-corpus/2-data/1-fr/'
    path = path + acteur + '/'

os.chdir(path)
len(os.listdir())

4342

In [75]:
corpus = []

for file in os.listdir():
    if file.endswith(".txt") and not file.endswith('-corpus_FR.txt') and not 'PDF' in file:
        file_path = path + file
        
        with open(file_path, 'r', encoding = "UTF-8") as f:
            data = f.readlines()
            text = re.sub('\d', '', data[1].strip('\n').lower().replace('’', '\''))
            text = text.replace('  ', ' ')
            corpus.append(text)

In [76]:
count_pdf = 0
for file in os.listdir():
    if 'PDF' in file:
        count_pdf +=1

print('Il y avait {} documents PDF dans notre dossier, pour l\'instant on ne les traitera pas.'.format(count_pdf))

Il y avait 0 documents PDF dans notre dossier, pour l'instant on ne les traitera pas.


In [77]:
corpus = corpus[:round(len(corpus))]

nb_docs = len(corpus)

print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 4342 documents.


### **Segmentation** (phrases)

**NLTK**  
https://www.nltk.org/ 

In [78]:
import nltk
#nltk.download(['popular'])

In [79]:
from nltk import sent_tokenize 

sents = [[s.strip('.') for s in sent_tokenize(doc)] for doc in corpus]

*Il faudrait probablement modifier le script pour rammener le POS tagging et la lemmatisation ici (TreeTagger tokenise lui-même déjà)*

### **Tokenisation**

In [80]:
from nltk.tokenize import RegexpTokenizer

# Seulement les caractères alphabétiques
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")

In [81]:
tokens = [[tokenizer_re.tokenize(s) for s in doc] for doc in sents]

len_corpus = len(nltk.flatten(tokens))

In [82]:
print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len_corpus))

Avec le RegExpTokenizer, notre corpus contient 39223816 tokens.


### **Filtrage** (antidictionnaire)

In [83]:
# Importer l'antidictionnaire pour filtrer les données
from pandas import *

# Stopwords fréquents en français
path = "/Users/camilledemers/Documents/04-filtrage/stopwords.csv"
with open(path, 'r', encoding="utf-8") as f:
    stopwords = read_csv(f)
    stopwords = [t.lower() for t in stopwords['Stopwords'].tolist()]


# Stopwords fréquents en anglais
path = '/Users/camilledemers/Documents/04-filtrage/stop_words_english.txt'
with open(path, 'r', encoding="utf-8") as f:
    sw = [w.strip('\n').lower() for w in f.readlines()]

stopwords += sw

# Signes de ponctuation
import string 
punct = [s for s in string.punctuation] 
punct += ['»' ,'©', '']

stopwords += punct


# Mis en commentaire pour l'instant car ça allonge le délai de traitement
#Prénoms (curieusement, il y en a beaucoup dans les données)
#path = '/Users/camilledemers/Documents/04-filtrage/Prenoms.csv'
#with open(path, 'r', encoding='utf-8') as f:
#     sw = read_csv(f)
#     sw = [str(t).lower() for t in sw['01_prenom'].tolist()]

#stopwords += sw

#Noms de famille 
# path = '/Users/camilledemers/Documents/04-filtrage/nomsFamille.csv'
# with open(path, 'r', encoding='utf-8') as f:
#     sw = read_csv(f)
#     sw = [str(t).lower() for t in sw['Nom'].tolist()]

# stopwords += sw


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [84]:
path = '/Users/camilledemers/Documents/04-filtrage/mwe_stopwords.txt'

with open (path, 'r', encoding='utf-8') as f:
    mwe_sw = [tuple(tokenizer_re.tokenize(t)) for t in f.readlines()]
    #mwe_sw = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [85]:
from nltk.tokenize import MWETokenizer
tokenizer_mwe = MWETokenizer(mwe_sw, separator=' ')

In [86]:
mwe_sw = [tokenizer_mwe.tokenize(w)[0] for w in mwe_sw]

In [87]:
tokens = [[[t for t in tokenizer_mwe.tokenize(sent) if t not in mwe_sw] for sent in doc] for doc in tokens]

### **Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook

In [88]:
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.probability import FreqDist

In [89]:
from nltk.util import everygrams
ngrammes = [[list(everygrams(sent, min_len=2, max_len=4)) for sent in doc] for doc in tokens]

### **Filtrage (N-grammes)**

On retire les n-grammes qui apparaissent moins de 30 fois dans tout le corpus ou qui débutent ou terminent par :
- un stopword
- un mot de 1 lettre ou moins

Pour le reste du traitement, on arrête de considérer les frontières entre les phrases et entre les documents (nos ngrammes les respectent donc on n'en a plus besoin)

In [90]:
# Calculer la distribution de fréquence de chaque ngramme dans tout le corpus
# Pour ça, on va 'applatir' la liste des ngrammes pour ne plus tenir compte des frontières entre les phrases et entre les documents
def ng_flat(ngramme):
    liste    = [] 
    for doc in ngrammes:
        for sent in doc:
            for ngram in sent:
                liste.append(ngram)
    return liste

ngrammes = ng_flat(ngrammes)
freq = FreqDist(ngrammes)

In [91]:
len(ngrammes)

114960688

In [92]:
import pandas as pd, re

ngrammes = [ngram for ngram in ngrammes if freq[ngram] > 30 and \
        not ngram[0] in stopwords and len(ngram[0]) > 2 \
        and not ngram[-1] in stopwords and len(ngram[-1]) > 2 \
        and not (ngram[0] == ngram[-1])]

In [93]:
len(ngrammes)

55790845

In [94]:
phrases = [" ".join(ngram).replace('\' ', '\'') for ngram in ngrammes]
freq = FreqDist(phrases)

In [95]:
vocabulaire = freq.keys()

def tabCSV(tab):
    tab = DataFrame(tab.items(), columns= ["Expression", "Fréquence"])
    tab.sort_values(["Fréquence"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)


    #path = '/Users/camilledemers/Documents/04-filtrage/' + acteur + '/'
    #if sous_corpus:
    #    path += tag + '/'
    #    file_path = path + acteur + '_' + tag
        
    
    #else:
    #    file_path = path 

    Path(path).mkdir(parents=True, exist_ok=True)

    return tab

In [96]:

path = '/Users/camilledemers/Documents/04-filtrage/' 

if sous_corpus:
    path = path + acteur + '/' + tag + '/'
    titre =   tag
else :
    path = path + acteur + '/'
    titre = acteur

tab = tabCSV(freq)

tab.to_csv(path + titre + '_n-grams.csv')

### **POS Tagging** (TreeTagger)
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [97]:
import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG='fr')

### **Mapping POS Tags** (FRMG)

Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

http://alpage.inria.fr/frmgwiki/content/tagset-frmg

In [98]:
path = '/Users/camilledemers/Documents/04-filtrage/mapping_treeTagger_lefff.csv'

with open(path) as f:
    csv = read_csv(f)

treeTag = [term for term in csv['TreeTagger'].tolist()] 
lefff = [term for term in csv['Lefff'].tolist()]

mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [99]:
tagged = [[phrase, " ".join([mapping[t.split('\t')[1]] for t in tagger.tag_text(phrase)])] for phrase in phrases]

### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (path, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:50] # Pour prendre seulement les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [ ]:
terms = [t for t in tagged if t[1] in patterns]

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage/'
tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique"])
tab = pd.DataFrame(tab.groupby(["Expression", "Structure syntaxique"]).size().reset_index(name="Fréquence"))
tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

if sous_corpus:
    path = path + acteur + '/' + tag + '/'
    titre =   tag
else :
    path = path + acteur + '/'
    titre = acteur

                    
tab.to_csv(path + titre + '_phrases.csv')

### **Lemmatisation** (FrenchLefffLemmatizer)

https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

In [ ]:
input = [] 
for term in terms:
    exp = tokenizer_re.tokenize(term[0]) # Un seul token dans un ngramme
    pos = term[1].split() # Étiquette morphosyntaxique

    term = [exp, pos]

    n = len(exp) # == len(exp[1])

    extr = []
    for i in range(n):
        try:
            extr.append((term[0][i], term[1][i]))
        except:
            print(" ".join(term[0]))

    input.append(extr)

In [ ]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

In [ ]:
lemmatizer = FrenchLefffLemmatizer()

In [ ]:
lemmas = []

for term in input:
    term_lemmatized = []
    for t in term:
        if(lemmatizer.lemmatize(t[0], t[1]) == []):
            term_lemmatized.append(lemmatizer.lemmatize(t[0]))
        else:
            term_lemmatized.append(lemmatizer.lemmatize(t[0], t[1])[0][0]) # [0][0] pour avoir le lemme seul et non (lemme, pos)
    
    lemmas.append(term_lemmatized)

In [ ]:
lemmas = [" ".join(t) for t in lemmas if len(t[0]) > 1 and len(t[-1]) > 1]

In [ ]:
freq = FreqDist(lemmas)
tab = pd.DataFrame(freq.items(), columns = ["Expression lemmatisée", "Fréquence"])
tab.sort_values(["Fréquence"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)
                    
path = '/Users/camilledemers/Documents/04-filtrage/' 
if sous_corpus:
    path = path + acteur + '/' + tag + '/'
    titre =   tag
else :
    path = path + acteur + '/'
    titre = acteur


tab.to_csv(path + titre + '_phrases_lemmatized.csv')

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne']

In [ ]:
extrant = pd.DataFrame(columns=['Mot-clé','Concordance', 'Fréquence'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in phrases: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df = pd.DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence"])
    df.sort_values(["Fréquence"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df.insert(0, 'Mot-clé', term)
    extrant = pd.concat([extrant, df])

path = '/Users/camilledemers/Documents/04-filtrage' + '/'
if sous_corpus:
    path = path + acteur + '/' + tag + '/'
    titre =   tag
else :
    path = path + acteur + '/'
    titre = acteur


extrant.to_csv(path + titre + '_KWIC' +'.csv')

### **Filtrage (fréquence)**
Pour la suite du traitement, on ne retient que les N expressions (lemmatisées ou non) les plus fréquentes dans le corpus

In [ ]:
# Sans lemmatiser
freq_terms = FreqDist([t[0] for t in terms])
exp_freq = freq_terms.most_common(1000) # On garde un maximum de 1000 termes

path = '/Users/camilledemers/Documents/05-transformation/' + acteur + '/'

if sous_corpus:
    path = path + tag + '/'
    titre =   tag
else :
    titre = acteur

                    
Path(path).mkdir(parents=True, exist_ok=True)
tab = pd.DataFrame([t[0] for t in exp_freq], columns=["Terme"])
tab.to_csv(path + titre + '_terms.csv')

# En lemmatisant
freq_terms = FreqDist([t for t in lemmas])
exp_freq = freq_terms.most_common(1000) # On garde un maximum de 1000 termes

path = '/Users/camilledemers/Documents/05-transformation/' + acteur + '/'

if sous_corpus:
    path = path + tag + '/'
    titre =   tag
else :
    titre = acteur

                    
Path(path).mkdir(parents=True, exist_ok=True)
tab = pd.DataFrame([t[0] for t in exp_freq], columns=["Terme"])
tab.to_csv(path + titre + '_terms-lemmatized.csv')

### **Extraction de termes MeSH**

In [ ]:
from nltk.tokenize import MWETokenizer
path = '/Users/camilledemers/Documents/04-filtrage/MeSH/mesh-fr.txt'

with open (path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
freq_terms = FreqDist([t[0] for t in terms])
exp_freq = freq_terms.most_common()

extr_mesh = tokenizer_mesh.tokenize([t[0] for t in exp_freq])

In [ ]:
termes_mesh = []

for t in extr_mesh:
    if t in mesh:
        termes_mesh.append(t)

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage' + '/'
if sous_corpus:
    path = path + acteur + '/' + tag + '/'
    titre =   tag
else :
    path = path + acteur + '/'
    titre = acteur

df = DataFrame(termes_mesh)

df.to_csv(path + titre + '_MeSH.csv')

### **Extraction de termes SNOMED**

In [ ]:
from nltk.tokenize import MWETokenizer
path = '/Users/camilledemers/Documents/04-filtrage/SNOMED/SNOMED_fr.csv'

with open(path, 'r', encoding='utf-8') as f:
    sm = read_csv(f, sep=';')
    sm = list(dict.fromkeys([str(t).strip().lower() for t in sm['term'].tolist()]))

    sm = [tuple(tokenizer_re.tokenize(w)) for w in sm if len(w.split()) > 1]
    tokenizer_sm = MWETokenizer(sm, separator = ' ')

    sm = [tokenizer_sm.tokenize(w)[0].lower() for w in sm]

In [ ]:
extr_sm = tokenizer_sm.tokenize([t[0] for t in exp_freq])

In [ ]:
termes_sm = []

for t in extr_sm:
    if t in sm:
        termes_sm.append(t)

In [ ]:
path = '/Users/camilledemers/Documents/04-filtrage' + '/'
if sous_corpus:
    path = path + acteur + '/' + tag + '/'
    titre =   tag
else :
    path = path + acteur + '/'
    titre = acteur

df = DataFrame(termes_sm)

df.to_csv(path + titre + '_SNOMED.csv')